<a href="https://colab.research.google.com/github/pascarujo/assistente-sage-xvii-stpc/blob/main/notebooks/sage_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning do Llama para Assistente Sage

## 1. Instalando as bibliotecas necessárias

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

## 2. Configuração inicial

In [2]:
model_name = "unsloth/Meta-Llama-3.1-8B"
#max_seq_length = 16384
max_seq_length = 2048
batch_size = 1
epochs = 4
learning_rate = 2e-5

In [ ]:
from unsloth import FastLanguageModel
import torch

dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

## 3. Carregando os dados

#### Carregando os exemplos QA do Hugging Face

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset
import pandas as pd
import json

In [5]:
# Carregar a base de treinamento do Hugging Face
data_path = 'pascarujo/sage-qa-training'

rawdata = load_dataset(data_path)

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

qa_training_data.jsonl:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1740 [00:00<?, ? examples/s]

In [ ]:
rawdata['train']['messages'][0]

[{'role': 'system',
  'content': 'Você é um assistente especialista no sistema SAGE do Cepel.'},
 {'role': 'user', 'content': 'Qual a estrutura de um PDS?'},
 {'role': 'assistant',
  'content': 'Estrutura de um PDS (Ponto Digital) no SAGE\nUm PDS, ou Ponto Digital, no sistema SAGE é uma entidade que representa um ponto de medição digital lógico. Ele armazena informações sobre o estado de um dispositivo digital, como um disjuntor ou chave seccionadora, que pode estar em um dos dois estados: aberto (0) ou fechado (1).\nA estrutura de um PDS é definida por diversos atributos, que podem ser divididos em:\n\n1. Identificação:\nID: Identificador único do PDS.\nNOME: Descrição do ponto por extenso.\nNOPO: Número operacional do equipamento.\nIDOPER: Identificador específico para o operador.\nIDICCP: Identificador para Servidor ICCP.\n\n2. Configuração:\nTIPO: Tipo do equipamento onde se localiza a medida digital (ex: DISJ, CHAVE).\nTPEQP: Tipo do equipamento associado ao ponto digital (ex: CNC

### Formatação dos dados em instruction, input, output

In [6]:
instructions = []
responses = []

# Percorrer os dados e acessar a lista dentro de 'messages'
for entry in rawdata['train']:
    messages = entry['messages']
    current_instruction = ""

    # Iterar sobre cada mensagem dentro de 'messages'
    for message in messages:
        if message['role'] == 'user':
            current_instruction = message['content']
        elif message['role'] == 'assistant' and current_instruction:
            instructions.append(current_instruction)
            responses.append(message['content'])
            current_instruction = ""


# Criar um DataFrame com as instruções e respostas
df = pd.DataFrame({
    "instruction": instructions,
    "input": ["" for _ in instructions],  # Como não há input adicional, manter vazio
    "output": responses
})

print(df.head())

# Transformar o DataFrame em um Dataset do Hugging Face
dataset = Dataset.from_pandas(df)

print(dataset)

                                         instruction input  \
0                        Qual a estrutura de um PDS?         
1         Qual a relação de um ponto PDS com um PDF?         
2  Faça um roteiro passo a passo de como incluir ...         
3  Faça um roteiro passo a passo de como incluir ...         
4  Qual a função do atributo KCONV na tabela PDF ...         

                                              output  
0  Estrutura de um PDS (Ponto Digital) no SAGE\nU...  
1  Um ponto PDS (Ponto Digital Lógico) e um PDF (...  
2  Roteiro passo a passo para incluir um novo CNF...  
3  Alterações na base de dados para incluir uma n...  
4  O atributo KCONV na tabela PDF define a conver...  
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1740
})


In [ ]:
# Salvar o dataset em disco para uso posterior (opcional)
dataset.save_to_disk('dataset')

Saving the dataset (0/1 shards):   0%|          | 0/1740 [00:00<?, ? examples/s]

In [ ]:
dataset = Dataset.load_from_disk('dataset')

In [7]:
def formatting_prompts_func(examples):
    alpaca_prompt = """Abaixo há uma instrução que descreve uma tarefa,
    acompanhada por uma entrada que fornece contexto adicional.
    Escreva uma resposta que complete de forma apropriada a solicitação.

### Instrução:
{}

### Entrada:
{}

### Resposta:
{}"""

    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + " EOS_TOKEN"
        texts.append(text)

    return {"text": texts}

# Aplicar a função de formatação
dataset = dataset.map(formatting_prompts_func, batched=True)

# Verificar os dados formatados
print(dataset[0])

Map:   0%|          | 0/1740 [00:00<?, ? examples/s]

{'instruction': 'Qual a estrutura de um PDS?', 'input': '', 'output': 'Estrutura de um PDS (Ponto Digital) no SAGE\nUm PDS, ou Ponto Digital, no sistema SAGE é uma entidade que representa um ponto de medição digital lógico. Ele armazena informações sobre o estado de um dispositivo digital, como um disjuntor ou chave seccionadora, que pode estar em um dos dois estados: aberto (0) ou fechado (1).\nA estrutura de um PDS é definida por diversos atributos, que podem ser divididos em:\n\n1. Identificação:\nID: Identificador único do PDS.\nNOME: Descrição do ponto por extenso.\nNOPO: Número operacional do equipamento.\nIDOPER: Identificador específico para o operador.\nIDICCP: Identificador para Servidor ICCP.\n\n2. Configuração:\nTIPO: Tipo do equipamento onde se localiza a medida digital (ex: DISJ, CHAVE).\nTPEQP: Tipo do equipamento associado ao ponto digital (ex: CNC, LTR).\nEST: Estação a que pertence o ponto digital.\nTAC: TAC (Terminal de Aquisição e Controle) à qual o ponto pertence.\

In [ ]:
# Salvar o dataset formatado
dataset.save_to_disk('formatted_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/1740 [00:00<?, ? examples/s]

In [8]:
# Dividir os dados em um conjunto de treinamento e avaliação (opcional)

datasets = dataset.train_test_split(test_size=0.1)
train_dataset = datasets['train']
eval_dataset = datasets['test']

## 4. Confirgurando o LoRA e o treinamento

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_rslora = True,
    use_gradient_checkpointing = 'unsloth',
    random_state = 0,
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
project = "Meta-Llama-3.1-8B-bnb-4bit-01"
run_name = "run_1_SS" # Defining a separate run name in case you want to start another one resuming from a checkpoint
project_and_run_name = project + "-" + run_name
output_dir = "./" + project_and_run_name

## 5. Treinamento

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from datetime import datetime
from unsloth import is_bfloat16_supported

wandbname = project + "-" + run_name

trainer = SFTTrainer(
    model = model,
    #callbacks=[upload_checkpoint_callback],
    train_dataset = dataset,
    #train_dataset = train_dataset,
    #eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    max_seq_length = max_seq_length,
    packing = True,
    dataset_text_field="text",
    args = TrainingArguments(
        #per_device_eval_batch_size = 1,
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = 2,
        warmup_ratio = 0,
        max_grad_norm = 1.0,
        num_train_epochs = epochs,
        learning_rate = learning_rate,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        #evaluation_strategy="epoch",
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "cosine",
        save_strategy="epoch",
        seed = 3407,
        output_dir = output_dir,
        run_name=f"sagista-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
    ),
)



Generating train split: 0 examples [00:00, ? examples/s]

### Treinar modelo

In [ ]:
trainer.train()

# If resuming from a checkpoint:
# trainer.train(resume_from_checkpoint="/content/latest-checkpoint/")

## 6. Testando o modelo treinado


In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
    """Explique os campos de uma tabela PDS"""
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 8192,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()

length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

### 7. Salvando modelo no Google Drive e Hugging Face

In [ ]:
# fazendo o merge do lora e salvando o modelo ajustado
model.save_pretrained_merged("sagista-01", tokenizer, save_method = "merged_16bit",)
!cp -r "/content/sagista-01/" "/content/drive/My Drive/sagista-01/"

### Enviando modelo pro Hugging Face

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained("/content/drive/My Drive/sagista-01/")

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF')
model.push_to_hub_merged("pascarujo/SageLlama-3.1-8B", tokenizer, save_method="merged_16bit", token=hf_token)

In [ ]:
quant_methods = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"]
for quant in quant_methods:
    model.push_to_hub_gguf("pascarujo/SageLlama-3.1-8B-GGUF", tokenizer, quant, token=hf_token)
